<a href="https://colab.research.google.com/github/SohailLab/PSI_redChl/blob/main/PSI_redChl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import scipy.io
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
exp_data = scipy.io.loadmat('/content/drive/My Drive/experimental_traces.mat');
traces = exp_data['exp_data']
time = exp_data['time']

In [ ]:
def Trimeric_and_Monomeric_PSI_monomersREV(exc_per_monomer,hopP700, bulkFluor, hopRed,hopIntra, iter, pct_trimer,pct_abs_red, pct_abs_bulk ):


  dt = 0.01 #time step in ps
  t = np.arange(0,10,dt)  #time range in ps
  max_q = iter
  max_i = len(t)
  max_p = 4

  annihilationvec_prox = np.zeros((max_q,max_i))
  annihilationvec_distal = np.zeros((max_q,max_i))
  RCtrapvec = np.zeros((max_q,max_i))
  totBulk = np.zeros((max_q,max_i))
  totRed = np.zeros((max_q,max_i))
  Red_results = np.zeros((max_i,max_p))
  Bulk_results = np.zeros((max_i,max_p))
  annivec_mean_prox = np.zeros((max_i,max_p))
  annivec_mean_distal = np.zeros((max_i,max_p))

  cols = ['black','red','cyan','green'];

  #This accounts for sole monomers and monomers in trimers absorbing differently at the red Chls as is expected with different numbers of red sites
  abs_red = pct_abs_red/(2*(1-pct_trimer)+3*pct_trimer)

  for p in range(0,4):


    for q in range(0,iter):
      trimer =  np.array([1,4,4,0,2,2,2,2,2,2,2,2,2,2,2,2])
      monomer = np.array([1,4,2,0,2,2,2,2,2,2,2,2,2,2,2,2])

        # 4 is proximal red Chl
        # 1 is distal red Chl
        # 2 is bulk Chl
      random_number = np.random.rand(1)
      if random_number>pct_trimer:
        oligomerization_state = 1
      else:
        oligomerization_state =3

      if oligomerization_state==3:
        assembly = trimer
      else:
        assembly  = monomer

      #find grid locations of each species
      red = np.array(np.where( assembly == 4))
      blanks = np.array(np.where (assembly==0))
      allpigments = np.array(np.where(assembly!=0))
      red_1 = np.array(np.where(assembly==1))
      bulk = np.array(np.where(assembly==2))

      power_R = abs_red*exc_per_monomer[p]#excitations per pigment, for proximal red Chls
      power_1 = abs_red*exc_per_monomer[p]; # excitations per pigment, for distal red Chls
      power_B = (pct_abs_bulk*exc_per_monomer[p])/np.shape(bulk)[1];#excitations per pigment, for bulk Chl sites

      array = assembly*0 #start with no excitations
      array[blanks]=-1

      dist =np.random.rand(1,np.shape(red)[1])
      array[red[np.where(dist<(power_R))]]=1

      dist1 =np.random.rand(1,np.shape(red_1)[1])
      array[red_1[np.where(dist1<(power_1))]]=1

      dist2 = np.random.rand(1,np.shape(bulk)[1])
      array[bulk[np.where(dist2<(power_B))]]=1

      array = assembly*0 #start with no excitations
      array[blanks]=-1

      dist =np.random.rand(1,np.shape(red)[1])
      array[red[np.where(dist<(power_R))]]=1

      dist1 =np.random.rand(1,np.shape(red_1)[1])
      array[red_1[np.where(dist1<(power_1))]]=1

      dist2 = np.random.rand(1,np.shape(bulk)[1])
      array[bulk[np.where(dist2<(power_B))]]=1

      totexcited = np.sum(np.sum(array>0))
      foo=[]
      annihilation_prox_reds=0
      annihilation_bulk_reds=0
      RCtrap=0
      fluortot=0

      #array is randomly populated with excitations

      for i in range(0,len(t)):
        ind=np.where(array==1)
        ind = np.array(ind)
        dist=np.random.rand(1,np.shape(ind)[1])

        for elem in range(0,np.shape(ind)[1]):
          nn = np.zeros((1,1))
          nn[0]=3
          tup = ind[0][elem]
          if (assembly[tup]==2 )| (assembly[tup]==4):
            if assembly[tup]==2:
              if (tup==4) | (tup==5) |(tup==6)|(tup==7):
                nn[0]=0
              elif (tup ==8) |(tup==9)|(tup==10)|(tup==11):
                nn[0]=1
              elif (tup==12) |(tup==13)|(tup==14)|(tup==15):
                if oligomerization_state==3:
                  nn[0]=2
            elif assembly[tup]==4:
              if tup==1:
                nn[0]=2
              else:
                nn[0]=1

            nn[np.array(np.nonzero(nn)).transpose()[:,0]]
            nnvectype = np.zeros((len(nn),1))
            for k in range(0,len(nn)):
              ind1 = int(nn[k])

              nnvectype[k]=nnvectype[k]+np.sum(assembly[ind1]==4)*4
              nnvectype[k]=nnvectype[k]+np.sum(assembly[ind1]==1)


            probs = np.zeros(np.shape(nn))
            if assembly[tup]==2:  # if excitation is on bulk  Chl
              probs[0]=dt/hopRed

              foo=np.cumsum(probs)
              probs=foo*np.ceil(probs).transpose()
              if (dist[0][elem]<np.max(probs)) & (nn[0]!=3):
                hop=np.where(np.random.rand(1)*np.max(probs)<probs)
                hop = hop[1][0]
                array[tup]=0 #deexcite the complex you are on
                array[int(nn[hop])]=array[int(nn[hop])]+1 #excite the hop

            if (assembly[tup]==4 ): # if excitation is on a proximal red chl
              probs[np.array(np.where(nnvectype==1))]=0
              probs[np.array(np.where(nnvectype==2))]=0
              probs[np.array(np.where(nnvectype==3))]=0
              probs[np.array(np.where(nnvectype==4))]=dt/hopIntra
              probs[np.array(np.where(nnvectype==5))]=0

              foo=np.cumsum(probs)
              probs=foo*np.ceil(probs).transpose()
              if (dist[0][elem]<np.max(probs))& (nn[0]!=3):
                hop=np.where(np.random.rand(1)*np.max(probs)<probs)
                hop = hop[1][0]
                array[tup]=0 #deexcite the complex you are on
                array[int(nn[hop])]=array[int(nn[hop])]+1 #excite the hop

            del nnvectype
            del probs
            del nn


        annihilation_prox_reds=annihilation_prox_reds+np.sum(array[red[np.where(array[red]>1)]])-np.shape(np.where(array[red]>1))[1] # determine if there is annihilation at the proximal red Chls
        annihilation_bulk_reds=annihilation_bulk_reds+np.sum(array[red_1[np.where(array[red_1]>1)]])-np.shape(np.where(array[red_1]>1))[1] # determine if there is annihilation at due to transfer from bulk to distal red Chl

        # determine if there is fluorescence
        ind=np.array(np.where(array>0)) #Find all the excitations
        dist=np.random.rand(1,np.shape(ind)[1]) # determine if they fluoresce
        fluortot=fluortot+np.shape(np.where(dist<(dt/bulkFluor)))[1]
        array[ind[np.where(dist<(dt/bulkFluor))]]=0

        # determine if there is transfer to P700
        ind=np.array(np.where(array>0)) # find the excitations
        dist=np.random.rand(1,np.shape(ind)[1]) # determine if they transfer
        hop_test = np.random.rand(1,np.shape(dist)[1])
        for el in range(0,np.shape(dist)[1]):
          if dist[0][el]<(dt/hopP700):
            RCtrap=RCtrap+1
            array[ind[np.where(dist[0][el]<(dt/hopP700))]]=0


        array[np.where(array>1)]=1
        annihilationvec_prox[q][i]=annihilation_prox_reds
        annihilationvec_distal[q][i]=annihilation_bulk_reds
        RCtrapvec[q][i]=RCtrap
        totBulk[q][i]=np.sum(array[bulk]>0)
        totRed[q][i]=np.sum(array[red]>0)+np.sum(array[red_1]>0)


    temp=np.mean(totRed,axis=0)
    temp2=np.mean(totBulk,axis=0);
    Red_results[:,p]=np.mean(totRed,axis=0)
    Bulk_results[:,p]=np.mean(totBulk,axis=0)
    annivec_mean_prox[:,p] = np.mean(annihilationvec_prox,0)
    annivec_mean_distal[:,p] = np.mean(annihilationvec_distal,0)

    plt.plot(t*1000,Red_results[:,p]/Red_results[40,p],color=cols[p])
    plt.plot(t*1000,Bulk_results[:,p],color=cols[p],linestyle='dashed')

  plt.plot(time[0],traces[33:,0])
  plt.plot(time[0],traces[33:,1])
  plt.plot(time[0],traces[33:,2])
  plt.plot(time[0],traces[33:,3])
  plt.show()
  return Red_results, annivec_mean_prox, annivec_mean_distal, Bulk_results

In [ ]:
results, annivec_prox, annivec_distal, bulkExc = Trimeric_and_Monomeric_PSI_monomersREV([8.1,15.2,28.8,45.7],80, 5000, 0.125,3.2, 5000, 0.6,0.05, 0.15 )

In [ ]:
dt = 0.01 #time step in ps
t = np.arange(0,10,dt)
cols = ['black','red','cyan','green']

for p in range(0,4):
  plt.plot(t*1000,results[:,p]/results[40,p],color=cols[p])
  plt.plot(time[0],traces[33:,p])

plt.xlim((0,10000))
plt.ylim((0.3,1.1))